In [3]:
import sys
import os

# Get the current working directory (where the notebook is located)
current_dir = os.getcwd()

# Calculate the parent directory (two levels up)
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add the parent directory to sys.path if it's not already there
if parent_dir not in sys.path:
    sys.path.append(parent_dir)



import torch
from torch import nn, optim
from modules.transformers import GPT2
from transformers import GPT2Tokenizer
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader


/home/uzair/Projects/Transformers/GPT2/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

dm=128
dk=16
N=128
h=8
n_layers =4
vocab_size=tokenizer.vocab_size
model = GPT2(dm=dm, dk=dk, N=N, h=h, vocab_size=vocab_size, n_layers=n_layers)
model = model.cuda() if torch.cuda.is_available() else model

# Load the model
model_save_path = "/home/uzair/Projects/Transformers/GPT2/gpt2_tinystories.pth"
loaded_model = GPT2(dm=dm, dk=dk, N=N, h=h, vocab_size=vocab_size, n_layers=n_layers)
loaded_model.load_state_dict(torch.load(model_save_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
loaded_model = loaded_model.cuda() if torch.cuda.is_available() else loaded_model
loaded_model.eval()  # Important: Set the model to evaluation mode
print(f"Model loaded from {model_save_path}")


Model loaded from /home/uzair/Projects/Transformers/GPT2/gpt2_tinystories.pth


/tmp/ipykernel_1628334/2072540445.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load(model_save_path, map_location=torch.device('cu

In [26]:
prompt = "There was this man you see"
input_ids = tokenizer.encode(prompt, return_tensors="pt").cuda()
print(input_ids.shape)
with torch.no_grad():
    logits = model(input_ids)
logits.shape

torch.Size([1, 6])


torch.Size([1, 6, 50257])

In [25]:
loaded_model.embeddings(input_ids).shape

torch.Size([1, 6, 128])

In [28]:
loaded_model.blocks[0].attn.Wqkv.weight.shape

torch.Size([384, 128])